### **<h1>Data SMP</h1>**

In [2]:
# Imports
import pandas as pd
import plotly.graph_objects as go

# Permission to access my Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FILE_PATH = "/content/drive/MyDrive/Doral_Energy/data/raw_data/smp.xlsx"

  

Mounted at /content/drive


# Load the data

In [3]:
df = pd.read_excel(FILE_PATH)
df

,תאריך,שעה,עלות שולית מערכתית כולל אילוצים,עלות שולית מערכתית ללא אילוצים
0,2021/01/01,00:00:00,117.24,113.80
1,2021/01/01,00:30:00,114.10,112.70
2,2021/01/01,01:00:00,111.19,111.19
3,2021/01/01,01:30:00,100.81,100.81
4,2021/01/01,02:00:00,110.02,110.02
...,...,...,...,...
33593,30/11/2022,21:30:00,102.44,129.85
33594,30/11/2022,22:00:00,102.45,175.90
33595,30/11/2022,22:30:00,102.45,122.00
33596,30/11/2022,23:00:00,102.45,112.77


# Change column names from Hebrew to English 


In [4]:
df.rename(columns={'תאריך': 'date', 'שעה': 'hour', 'עלות שולית מערכתית כולל אילוצים': 'smp_const', 'עלות שולית מערכתית ללא אילוצים': 'smp_no_const' }, inplace=True)
df.head()

,date,hour,smp_const,smp_no_const
0,2021/01/01,00:00:00,117.24,113.80
1,2021/01/01,00:30:00,114.10,112.70
2,2021/01/01,01:00:00,111.19,111.19
3,2021/01/01,01:30:00,100.81,100.81
4,2021/01/01,02:00:00,110.02,110.02


# Check the types

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33598 entries, 0 to 33597
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          33598 non-null  object 
 1   hour          33598 non-null  object 
 2   smp_const     33598 non-null  float64
 3   smp_no_const  33598 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.0+ MB


#Get some understanding about the range of SMP


In [6]:
df.describe()

,smp_const,smp_no_const
count,33598.000000,33598.000000
mean,118.587472,121.580089
std,38.976578,23.257673
min,22.000000,71.430000
25%,101.570000,105.300000
50%,108.730000,112.760000
75%,121.500000,129.025000
max,1421.990000,220.000000


# Check if there are missing values


In [7]:
df.isna().any(axis=1).sum()

0

# Are there any duplicates?


In [8]:
df.duplicated().sum()

146

In [9]:
df[df.duplicated()]

,date,hour,smp_const,smp_no_const
16750,2021/12/17,00:00:00,104.68,106.73
16751,2021/12/17,00:30:00,106.73,107.70
16752,2021/12/17,01:00:00,105.41,107.17
16753,2021/12/17,01:30:00,97.18,103.81
16754,2021/12/17,02:00:00,95.18,100.04
...,...,...,...,...
16987,2021/12/19,22:30:00,109.12,109.91
16988,2021/12/19,23:00:00,109.13,106.45
16989,2021/12/19,23:30:00,104.99,100.11
32063,30/10/2022,01:00:00,109.84,107.67


In [10]:
# Drop the rows
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
df

,date,hour,smp_const,smp_no_const
0,2021/01/01,00:00:00,117.24,113.80
1,2021/01/01,00:30:00,114.10,112.70
2,2021/01/01,01:00:00,111.19,111.19
3,2021/01/01,01:30:00,100.81,100.81
4,2021/01/01,02:00:00,110.02,110.02
...,...,...,...,...
33447,30/11/2022,21:30:00,102.44,129.85
33448,30/11/2022,22:00:00,102.45,175.90
33449,30/11/2022,22:30:00,102.45,122.00
33450,30/11/2022,23:00:00,102.45,112.77


In [11]:
# Get the names of the columns from index 2
column_names = list(df.columns.values[2:])
# Group the data by date and hour
grouped_df = df.groupby(['date', 'hour'])

mask = grouped_df[column_names].transform('nunique') > 1
filtered_df = df.loc[mask.any(axis=1)]

filtered_df 

,date,hour,smp_const,smp_no_const


No different values for the same date and hour

#Outliers Detection


In [13]:
fig = go.Figure()
fig.add_trace(go.Box(name='SMP With Constraints', y=df['smp_const'], boxmean='sd'))

fig.update_traces(
    marker=dict(
        color='rgb(8,81,156)',
        outliercolor='rgba(219, 64, 82, 0.6)',
        line=dict(
            outliercolor='rgba(219, 64, 82, 0.6)',
            outlierwidth=2)),
    line_color='rgb(8,81,156)'
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(name='SMP With No Constraints', y=df['smp_no_const(MW)'], boxmean='sd'))

fig.update_traces(
    marker=dict(
        color='rgb(8,81,156)',
        outliercolor='rgba(219, 64, 82, 0.6)',
        line=dict(
            outliercolor='rgba(219, 64, 82, 0.6)',
            outlierwidth=2)),
    line_color='rgb(8,81,156)'
)
fig.show()

In [14]:
def get_outliers(df, series):
  q1 = series.quantile(0.25)
  q3 = series.quantile(0.75)

  if q1*q3 == 0:
    iqr = abs(2*(q1+q3))
    toprange = iqr
    botrange = -toprange
  else:
    iqr = q3-q1
    toprange = q3 + iqr * 1.5
    botrange = q1 - iqr * 1.5

  outliers_top=df[series > toprange]
  outliers_bot= df[series < botrange]
  outliers = pd.concat([outliers_bot, outliers_top], axis=0)

  return (botrange, toprange, outliers)

In [16]:
botrange, toprange, outliers = get_outliers(df, df['smp_no_const'])
print(toprange)
print(botrange)

outliers

165.0375
69.45750000000001


,date,hour,smp_const,smp_no_const
227,2021/01/05,17:30:00,153.36,169.62
229,2021/01/05,18:30:00,153.36,170.15
706,2021/01/15,17:00:00,165.14,165.14
707,2021/01/15,17:30:00,165.14,165.14
857,2021/01/18,20:30:00,165.14,165.14
...,...,...,...,...
33443,30/11/2022,19:30:00,111.90,180.71
33444,30/11/2022,20:00:00,111.90,182.20
33445,30/11/2022,20:30:00,100.61,180.11
33446,30/11/2022,21:00:00,102.44,179.03


In [17]:
# Change hour and date types 
df['date'] = df['date'].astype('datetime64[ns]').dt.strftime('%d/%m/%Y')
df['hour'] = df['hour'].astype('datetime64[ns]').dt.strftime('%H:%M')

In [18]:
df

,date,hour,smp_const,smp_no_const
0,01/01/2021,00:00,117.24,113.80
1,01/01/2021,00:30,114.10,112.70
2,01/01/2021,01:00,111.19,111.19
3,01/01/2021,01:30,100.81,100.81
4,01/01/2021,02:00,110.02,110.02
...,...,...,...,...
33447,30/11/2022,21:30,102.44,129.85
33448,30/11/2022,22:00,102.45,175.90
33449,30/11/2022,22:30,102.45,122.00
33450,30/11/2022,23:00,102.45,112.77


# Save the changes to excel file


In [19]:
df.to_csv('data_smp.csv', index=False)